In [1]:
import csv
import pandas as pd

In [2]:
OCCURRENCES_CSV = 'https://raw.githubusercontent.com/benscott/biodiversity-informatics-101/master/occurrences.csv'
data = pd.read_csv(OCCURRENCES_CSV, sep='\t')

In [3]:
data.head()

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,2540360757,7d17a2b2-8751-41b5-ac9b-48effc1ea236,faa4630f-fc51-46f9-8886-62a27297b48e,Plantae,Tracheophyta,Liliopsida,Alismatales,Hydrocharitaceae,Elodea,Elodea nuttallii,...,NaN,NaN,CC_BY_NC_4_0,NaN,Recorder details held at BRERC,NaN,NaN,2019-12-30T23:05:41.872Z,NaN,INDIVIDUAL_COUNT_INVALID
1,2540355888,7d17a2b2-8751-41b5-ac9b-48effc1ea236,fac13a1c-3b0b-4dac-b755-424a09cfe0a0,Plantae,Tracheophyta,Liliopsida,Alismatales,Hydrocharitaceae,Elodea,Elodea nuttallii,...,NaN,NaN,CC_BY_NC_4_0,NaN,Recorder details held at BRERC,NaN,NaN,2019-12-30T23:04:58.520Z,NaN,INDIVIDUAL_COUNT_INVALID
2,2540353022,7d17a2b2-8751-41b5-ac9b-48effc1ea236,ea777c16-0ce1-4e4e-817d-849b586cfa23,Plantae,Tracheophyta,Liliopsida,Alismatales,Hydrocharitaceae,Elodea,Elodea nuttallii,...,NaN,NaN,CC_BY_NC_4_0,NaN,Recorder details held at BRERC,NaN,NaN,2019-12-30T23:05:40.830Z,NaN,INDIVIDUAL_COUNT_INVALID
3,2540351534,7d17a2b2-8751-41b5-ac9b-48effc1ea236,f197b87b-d1ab-457b-bf00-fdb73906faa1,Plantae,Tracheophyta,Liliopsida,Alismatales,Hydrocharitaceae,Elodea,Elodea nuttallii,...,NaN,NaN,CC_BY_NC_4_0,NaN,Recorder details held at BRERC,NaN,NaN,2019-12-30T23:05:39.588Z,NaN,INDIVIDUAL_COUNT_INVALID
4,2540350974,7d17a2b2-8751-41b5-ac9b-48effc1ea236,f3862a80-4f2f-4823-a18d-68bf91a59545,Plantae,Tracheophyta,Liliopsida,Alismatales,Hydrocharitaceae,Elodea,Elodea nuttallii,...,NaN,NaN,CC_BY_NC_4_0,NaN,Recorder details held at BRERC,NaN,NaN,2019-12-30T23:05:40.889Z,NaN,INDIVIDUAL_COUNT_INVALID
